In [1]:
import pandas as pd
import spacy

from tqdm.notebook import tqdm
tqdm.pandas()

compute_freq = True


Ce code réalise une lemmatization à partir d'un modèle de machine learning issu de spacy.
installation du modèle : 
> python -m spacy download fr_core_news_sm

Ensuite on supprime certains mots fréquents, remis en forme et trouvés grâce à :
> from nltk.probability import FreqDist

Les nombres sont aussi filtrés.

J'ai réalisé cette approche car un  cleaning manuel était trop long (code en n² où n est le nombre de mots total du corpus). Le défaut de cette aproche est que potentiellement est elle trop bourrine et certains mots spécifiques qui nous intéressent ont disparu.

Il faut valider par une approche lexico et LDA les résultats => cf le code analyse_lexico.qmd

# Opening data

In [2]:
fichier = "../data/intermediate/base_merged.csv"

df = pd.read_csv(fichier, sep=';', quotechar='"')

df.sample(5)

,ID ASSO,ID ARTICLE,contenu,Type de document,Auteur,Date,Titre,URL
398,13,130028,Róisín Murphy et la fatalité d’une position ra...,blog,TRADFEM,09/09/2023,Róisín Murphy et la fatalité d’une position ra...,https://tradfem.wordpress.com/2023/09/09/roisi...
292,20,200064,"Vues par notre sensibilité sociétale actuelle,...",Article d'opinion,NaN,NaN,NaN,https://cryforrecognition.be/fr/confusion-entr...
467,13,130097,"Déclaration de Mme Reem Alsalem, Rapporteuse s...",blog,TRADFEM,22/05/2023,"Déclaration de Mme Reem Alsalem, Rapporteuse s...",https://tradfem.wordpress.com/2023/05/22/12813/
419,13,130049,Trop de fierté annonce la chute : Pourquoi j’a...,blog,TRADFEM,20/07/2023,Trop de fierté annonce la chute : Pourquoi j’a...,https://tradfem.wordpress.com/2023/07/20/trop-...
203,6,60003,Depuis la découverte de “transgenres” dans plu...,Article d'opinion,NaN,NaN,NaN,https://laportelatine.org/formation/crise-egli...


# Cleaning and tokenisation

## Lemmatization with Spacy

We also do some first filtering with Spacy

In [3]:
# Load the French language model
nlp = spacy.load("fr_core_news_sm")

# Define your list of words to exclude
words_to_filter = [
    'le', 'la', 'les', 'de', 'un', 'à', 'être', 'suis', 'es', 'est', 'sont', 'été', 'étais', 'était', 'étaient', 'serai', 'seras', 'sera', 'seront',
    'et', 'en', 'avoir', 'ai', 'as', 'a', 'ont', 'eu', 'aurai', 'auras', 'aura', 'auront', 'avais', 'avait', 'avaient', 'que', 'pour',
    'dans', 'ce', 'il', 'qui', 'ne', 'en', 'sur', 'se', 'pas', 'plus',
    'par', 'je', 'avec', 'tout', 'faire', 'fais', 'fait', 'font', 'son', 'mettre', 'autre', 'on', 'mais',
    'nous', 'comme', 'ou', 'si', 'leur', 'y', 'dire', 'dis', 'dit', 'disent', 'elle', 'voir', 'devoir',
    'deux', 'même', 'pendant', 'aussi', 'vouloir', 'grand', 'mon', 'nouveau', 'aller',
    'venir', 'ceux', 'faire', 'quelque', 'trouver', 'donner', 'donne', 'donnes', 'donnent', 'aussi', 'autre',
    'tous', 'vieux', 'bon', 'voir', 'moins', 'trois', 'avant', 'sa', 'faire', 'contre',
    'abord', 'sous', 'ou', 'apporter', 'grand', 'ainsi', 'long', 'très', 'tout', 'avoir',
    'beau', 'chaque', 'peu', 'quoi', 'encore', 'aller', 'montrer', 'semaine', 'ainsi',
    'nuit', 'aussi', 'bien', 'deuxième', 'moins', 'tout', 'avoir', 'peu', 'nuit', 'ça',
    'mon', 'ma', 'mes', 'ton', 'ta', 'tes', 'son', 'sa', 'ses', 'notre', 'notre', 'nos',
      'votre', 'votre', 'vos', 'leur', 'leur', 'leurs',
    'non', '’', '.', ',', ':', 'est', 'des', "l'", "d'", "une", "d",
      "»", "«", "du", "qu", "au", "n", "s", "elle", ")", "(", "aux", 'c', 'cette',
        'un', 'ces','?', 'il', 'ils', 'j', "l", "-", "vous", "entre",
    "[", "]", "elles", '”', '“', "dont", "cela", ";", "m", "selon", "!", '–',
      "en", "autre", "ici", "ce", "ça", "cela", "ceci", "car", "ou", "où",
        "ni", "mais", "et", "donc", "parce", "que", "quand", "comment",
          "qui", "quoi",
    "à", "de", "pour", "sans", "sur","l","d","que","lui","n","s","c","j","me","alors","certain","chez","celer"
    "lequel","m","cas","tel","luire","après","souvent","aucun","toujours","également","jamais","hui","beaucoup","rien"
]

# Function to tokenize, filter, and lemmatize text, excluding numbers
def tokenize_filter_and_lemmatize(text):
    # Process the text using spaCy
    doc = nlp(text)
    # Extract lemmatized forms of the words, excluding specific tokens and numbers
    lemmatized = " ".join([token.lemma_ for token in doc if token.lemma_ not in words_to_filter and not token.is_digit])
    return lemmatized

# Apply the function to the 'contenu' column
df['lemmatized_contenu'] = df['contenu'].progress_apply(tokenize_filter_and_lemmatize)

# Print the dataframe to see the result
df.sample(5)

  0%|          | 0/488 [00:00<?, ?it/s]

,ID ASSO,ID ARTICLE,contenu,Type de document,Auteur,Date,Titre,URL,lemmatized_contenu
233,23,230008,"Dans un rapport parlementaire, deux députés dé...",Article,NaN,NaN,NaN,https://www.observatoirepetitesirene.org/ruben...,rapport parlementaire député dénoncer stéréoty...
276,20,200048,Il croyait en l’auto-identification avant que ...,Article d'opinion,NaN,NaN,NaN,https://cryforrecognition.be/fr/a-quel-point-n...,croire l’ auto-identification mode \r\n n’ nie...
411,13,130041,Entrevue de Kajsa Ekis Ekman à propos de son n...,blog,TRADFEM,06/08/2023,Entrevue de Kajsa Ekis Ekman à propos de son n...,https://tradfem.wordpress.com/2023/08/06/entre...,entrevue Kajsa Ekis Ekman propos ouvrage l’ ex...
345,31,310004,Homme et femme Il les créa !\r\n\r\nLe monde m...,Article d'opinion,NaN,2021-02-01 00:00:00,NaN,https://www.civitas-institut.com/2021/02/01/la...,homme femme créa \r\n\r\n monde moderne préten...
330,30,300011,"\r\nLe 14 juillet dernier, à 20h22, Trystan, p...",Article d'opinion,NaN,2018-07-17 00:00:00,NaN,https://www.parti-de-la-france.fr/articles/On-...,\r\n juillet dernier 20h22 trystan père transg...


## Checking Words frequency according to Spacy tokenization

In [4]:
from collections import Counter
import spacy

if compute_freq:
    # Load the French language model
    nlp = spacy.load("fr_core_news_sm")

    # Your list of words to filter and the tokenize_filter_and_lemmatize function should be defined here

    # Initialize a Counter object to hold the frequency of each token
    token_freq = Counter()

    # Function to update token frequency for a single document
    def update_token_frequency(text):
        global token_freq  # Reference the global Counter object
        # Process the text with spaCy
        doc = nlp(text)
        # Update the Counter with tokens from this document, excluding filtered words and numbers
        token_freq.update([token.text for token in doc if token.lemma_ not in words_to_filter and not token.is_digit])

    # Apply the function to each row in the 'lemmatized_contenu' column to update the global token frequency
    df['lemmatized_contenu'].progress_apply(update_token_frequency)

    # Find the top 10 most frequent tokens
    top_10_tokens = token_freq.most_common(10)

    # Print the top 10 most frequent tokens
    print("Top 10 most frequent tokens:")
    for token, freq in top_10_tokens:
        print(f"{token}: {freq}")


  0%|          | 0/488 [00:00<?, ?it/s]

Top 10 most frequent tokens:
  : 7082
femme: 5329
pouvoir: 4187
genre: 3024

 : 2723
enfant: 2526
homme: 2447
personne: 2371
sexe: 2016

 : 2011


## Second filtering with nltk

the tokenization of spacy is better than the one I used in R (we do the LDA in R). The one in R is equivalent to the one of nltk. And for instance aujourd'hui appears as aujourd + ' + hui. So i need to run a second filter based on nltk tokenization.

In [5]:
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd

# Ensure NLTK resources are downloaded (needed for tokenization)
nltk.download('punkt')

# Function to tokenize with NLTK and filter based on your criteria
def nltk_tokenize_and_filter(text):
    # Tokenize the text with NLTK, ensuring the text is treated as French
    tokens = word_tokenize(text, language='french')
    # Filter tokens: convert to lowercase, exclude if in words_to_filter or is a digit
    filtered_tokens = [token.lower() for token in tokens if token.lower() not in words_to_filter and not token.isdigit()]
    # Join the filtered tokens back into a string
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

# Assuming 'df' is your DataFrame and 'lemmatized_contenu' contains the text to process
# Apply the function to each row in the 'lemmatized_contenu' column
df['lemmatized_contenu'] = df['lemmatized_contenu'].progress_apply(nltk_tokenize_and_filter)

# This results in a new column 'filtered_contenu' in your dataframe 'df' with the text processed as per your requirements


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\leopo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/488 [00:00<?, ?it/s]

## Checking words frequency based on nltk tokenization

In [6]:
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
import pandas as pd

if compute_freq:
    # Ensure you've downloaded the NLTK tokenizer models
    nltk.download('punkt')

    # Your DataFrame 'df' should already be loaded with the 'lemmatized_contenu' column ready

    # Initialize a Counter object for token frequencies
    token_freq = Counter()

    # Define your list of words to exclude, adjusted for your context
    words_to_filter = set([
        # Your list of words to filter
    ])

    # Function to tokenize and update token frequency for a single document using NLTK
    def update_token_frequency_nltk(text):
        global token_freq  # Reference the global Counter object
        # Tokenize the text using NLTK
        tokens = word_tokenize(text, language='french')
        # Update the Counter with tokens from this document, excluding filtered words and numbers
        token_freq.update([token.lower() for token in tokens if token.lower() not in words_to_filter and not token.isdigit()])

    # Apply the function to each row in the 'lemmatized_contenu' column to update the global token frequency
    df['lemmatized_contenu'].apply(update_token_frequency_nltk)

    # Find the top 10 most frequent tokens
    top_10_tokens = token_freq.most_common(10)

    # Print the top 10 most frequent tokens
    print("Top 10 most frequent tokens using NLTK tokenization:")
    for token, freq in top_10_tokens:
        print(f"{token}: {freq}")


NameError: name 'compute_freq' is not defined

# Export

In [4]:
fichier = "../data/intermediate/base_lemmatized.csv"
df.to_csv(fichier, sep=';', index=False, quotechar='"')